In [526]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
%matplotlib inline
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle

import requests

In [679]:
data = pd.read_csv('http://3e9387c7.ngrok.io/get_data')

In [680]:
data

,x,y,z


In [201]:
# data.to_csv('gesture_decline.csv')

In [667]:
gesture1 = pd.read_csv('gesture_accept.csv')
gesture2 = pd.read_csv('gesture_decline.csv')

#### training data prepocessing 

In [684]:
gesture1[:11]

,Unnamed: 0,x,y,z
0,0,0.000000,0.000000,0.000000
1,1,-0.064209,0.028076,0.154785
2,2,-0.645020,-0.220459,0.068604
3,3,2.791260,-0.025879,0.772705
4,4,-0.729736,-0.090332,0.064453
5,5,-0.746582,-0.261719,0.439941
6,6,3.005859,0.010254,-0.577637
7,7,-1.071533,-0.062988,0.036377
8,8,1.220459,0.320312,0.131836
9,9,2.091309,0.002686,-0.514893


In [671]:
a = gesture1.loc[:10]

In [662]:
gesture1.any(1)

0       False
1        True
2        True
3        True
4        True
        ...  
2996     True
2997     True
2998     True
2999     True
3000     True
Length: 3001, dtype: bool

In [666]:
(gesture1[:10] == 0).all(1)

0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [623]:
gesture1.loc[(gesture1==0).any(1)]

,Unnamed: 0,x,y,z
0,0,0.0,0.0,0.0
100,100,0.0,0.0,0.0
202,202,0.0,0.0,0.0
303,303,0.0,0.0,0.0
406,406,0.0,0.0,0.0
510,510,0.0,0.0,0.0
615,615,0.0,0.0,0.0
721,721,0.0,0.0,0.0
828,828,0.0,0.0,0.0
936,936,0.0,0.0,0.0


In [183]:
# 1 = accept, 2 = decline

gesture1 = gesture1.drop(columns=['Unnamed: 0'])
gesture2 = gesture2.drop(columns=['Unnamed: 0'])

drop_zeros = gesture1.loc[(gesture1==0).any(1)].index
gesture1 = gesture1.drop(drop_zeros, axis=0)
gesture1= gesture1.reset_index(drop=True)

drop_zeros = gesture2.loc[(gesture2==0).any(1)].index
gesture2 = gesture2.drop(drop_zeros, axis=0)
gesture2= gesture2.reset_index(drop=True)


gesture1['label'] = 0
gesture2['label'] = 1

In [185]:
gesture1

,x,y,z,label
0,-0.064209,0.028076,0.154785,0
1,-0.645020,-0.220459,0.068604,0
2,2.791260,-0.025879,0.772705,0
3,-0.729736,-0.090332,0.064453,0
4,-0.746582,-0.261719,0.439941,0
...,...,...,...,...
2985,2.043213,0.385010,-0.200928,0
2986,1.554932,-0.094482,-0.130371,0
2987,-0.460938,-0.301270,0.155762,0
2988,1.853760,0.293457,-0.322021,0


In [186]:
gesture1 = gesture1.loc[:2980]
gesture2 = gesture2.loc[:2470]

dataset = gesture1.append(gesture2)

dataset= dataset.reset_index(drop=True)

### Preprocessing 1

In [187]:
labels = dataset['label'].to_numpy()
labels = labels[:-9:10]

dataset = dataset[['x','y','z']]

In [188]:
dataset

,x,y,z
0,-0.064209,0.028076,0.154785
1,-0.645020,-0.220459,0.068604
2,2.791260,-0.025879,0.772705
3,-0.729736,-0.090332,0.064453
4,-0.746582,-0.261719,0.439941
...,...,...,...
5447,-0.494141,-0.026123,0.815186
5448,0.570068,-0.179443,0.455078
5449,2.004395,-0.074951,-0.950439
5450,0.312500,-0.072021,0.212891


In [189]:
labels.size

545

In [190]:
dataset = dataset.loc[:5450]

In [191]:
dataset

,x,y,z
0,-0.064209,0.028076,0.154785
1,-0.645020,-0.220459,0.068604
2,2.791260,-0.025879,0.772705
3,-0.729736,-0.090332,0.064453
4,-0.746582,-0.261719,0.439941
...,...,...,...
5446,1.616211,-0.020020,-1.242920
5447,-0.494141,-0.026123,0.815186
5448,0.570068,-0.179443,0.455078
5449,2.004395,-0.074951,-0.950439


### Preprocessing 2

In [192]:
frames = []
def frame():
    n=0
    while n != len(dataset)-1:
        frames.append(dataset.loc[n:n+1].to_numpy())
        n += 1
    else:
        return frames


In [193]:
gest_frames = frame()

In [194]:
len(gest_frames)

5450

### Peprocessing 3 > FFT, feature extraction and feature vector 

In [195]:
def fft_transform(frame_data):
    
    fft_transform.feature_set = np.array([])
    
    for frame in frame_data:
        
        frame_feature_set = np.array([])
        
        fft_frame = np.fft.fft(frame, axis=0)
#         fft_transform.fft_data.append(fft_frame)
        
#         n = frame.size
#         freq = np.fft.fftfreq(n, d=0.01)
#         fft_transform.fft_freq.append(freq)
        
#         fft_ps = np.abs(fft_frame)**2
#         fft_transform.fft_power_spec.append(fft_ps)
        
        mean = np.mean(fft_frame, axis=0)
        frame_feature_set = np.concatenate((frame_feature_set, mean), axis=0)
        
        
        energy = (np.sum((fft_frame**2),axis=0))/len(fft_frame)
        frame_feature_set = np.concatenate((frame_feature_set, energy), axis=0)
        
        
        stdev = np.std(fft_frame, axis=0)
        frame_feature_set = np.concatenate((frame_feature_set, stdev), axis=0)
        
        
        correlation = np.corrcoef(fft_frame, rowvar=False).reshape(3,3)
        frame_feature_set = np.concatenate((frame_feature_set.reshape(3,3), correlation), axis=0)
        
        fft_transform.feature_set = np.concatenate((frame_feature_set.flatten(), fft_transform.feature_set), axis=0)
    
#  make a simple array         

    return fft_transform.feature_set

In [556]:
fft_transform(gest_frames)

array([ 2.00439453+0.j, -0.07495117+0.j, -0.95043945+0.j, ...,
       -1.        +0.j, -1.        +0.j,  1.        +0.j])

### Training data set 

In [409]:
n = int((fft_transform.feature_set.size)/180)
axis = 3
feature_10axis = 60
fft_transform.feature_set = fft_transform.feature_set.reshape(n,(feature_10axis*axis))
X = np.abs(fft_transform.feature_set)
y = labels

In [410]:
y.shape

(545,)

In [551]:
X[1]

array([0.8215332 , 0.01879883, 0.42553711, 3.43184626, 0.02356201,
       0.58215171, 1.66040039, 0.15234375, 0.63330078, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.3737793 , 0.14208984,
       0.16918945, 0.81462777, 0.02054292, 0.2097069 , 0.8215332 ,
       0.01879883, 0.42553711, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.21655273, 0.30664062, 0.28100586, 1.61971152,
       0.114218  , 0.10758936, 0.3737793 , 0.14208984, 0.16918945,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.36889648,
       0.15087891, 0.82226562, 1.61608517, 0.11679292, 0.75508505,
       1.21655273, 0.30664062, 0.28100586, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.37353516, 0.12451172, 0.84106

In [541]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [542]:
C = 0.1
clf = svm.SVC(kernel='poly',C=C, probability=True)
clf.fit(X_train, y_train)

/Users/martinantos/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [543]:
with open('/Users/martinantos/Documents/school/IoT/Panache/clf.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [544]:
model = pickle.dumps(clf)

In [545]:
# clftest = pickle.loads(model)

In [546]:
# clftest.predict(X.reshape(1,-1))

In [547]:
y_pred = clf.predict(X_test)

In [548]:
print("accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

accuracy: 0.9541284403669725
Precision: 0.9056603773584906
Recall: 1.0


In [511]:
# 1 = accetpt, 
def clfout(sample):
    sample = sample.reshape(1,-1)
    if clf.decision_function(sample) <=(-1.2):
        return 1
    elif clf.decision_function(sample) >= 1.0:
        return 2 
    else:
        return 0

In [525]:
clfout(X[356])

2

In [510]:
clf.decision_function(X).max()

5.006824207538445

In [504]:
clf.decision_function(X)

array([-1.00013704, -1.36967167, -1.29377807, -2.00024957, -1.07263898,
       -1.02405042, -1.97569432, -2.05931219, -1.29023245, -1.42848515,
       -1.42992087, -0.8875324 , -1.71344709, -1.60367419, -1.90991305,
       -0.97405165, -1.97302585, -1.40037917, -1.36782399, -1.80207238,
       -1.56918316, -0.79874479, -1.52952697, -1.54467402, -2.10057753,
       -0.81397593, -2.17558273, -1.78192584, -1.88861074, -1.56609852,
       -1.05444061, -0.99994218, -1.53095592, -1.00038054, -1.77572437,
       -1.47920484, -1.28055973, -1.5533746 , -2.6023298 , -1.09390969,
       -2.36064849, -1.00013704, -1.36967167, -1.29377807, -2.00024957,
       -1.07263898, -1.02405042, -1.97569432, -2.05931219, -1.29023245,
       -2.27437666, -1.00013704, -1.36967167, -1.29377807, -2.00024957,
       -1.07263898, -1.02405042, -1.97569432, -2.05931219, -1.29023245,
       -1.93410539, -1.78060517, -2.00065274, -2.49952814, -2.33225689,
       -3.70645188, -2.78580244, -1.30455404, -2.7221272 , -2.12

In [405]:
clf.predict(X[65].reshape(1,-1))

array([0])

In [406]:

clf.decision_function(X[65].reshape(1,-1))

array([-2.35995993])